# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings_small.csv',low_memory=False)

In [3]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,original_title,revenue,runtime,id,budget,popularity,original_language,genres
0,Toy Story,373554033.0,81.0,862,30000000,21.946943,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,262797249.0,104.0,8844,65000000,17.015539,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,0.0,101.0,15602,0,11.7129,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,81452156.0,127.0,31357,16000000,3.859495,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,76578911.0,106.0,11862,0,8.387519,en,"[{'id': 35, 'name': 'Comedy'}]"


In [5]:
#Tirando budgets diferente de 0 (budget ainda como string
df=df[df.budget != "0"]
df=df.drop([19730, 29503,35587])

In [6]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

original_title        object
revenue              float64
runtime              float64
id                   float64
budget               float64
popularity            object
original_language     object
genres                object
dtype: object

In [7]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df = df.set_index("id")
df.head()

,original_title,revenue,runtime,budget,popularity,original_language,genres
id,,,,,,,
5.0,Four Rooms,4300000.0,98.0,4000000.0,9.026586,en,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name..."
11.0,Star Wars,775398007.0,121.0,11000000.0,42.149697,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '..."
12.0,Finding Nemo,940335536.0,100.0,94000000.0,25.497794,en,"[{'id': 16, 'name': 'Animation'}, {'id': 10751..."
13.0,Forrest Gump,677945399.0,142.0,55000000.0,48.307194,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
14.0,American Beauty,356296601.0,122.0,15000000.0,20.726578,en,"[{'id': 18, 'name': 'Drama'}]"


In [8]:
imdb.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [9]:
#Tirando media dos ratings
imdb=imdb.sort_values(['movieId'], ascending=[True])
imdb=imdb.rename(columns={'movieId': 'id'})
imdb=imdb.drop("userId",1)
imdb=imdb.drop("timestamp",1)
imdb.columns


Index(['id', 'rating'], dtype='object')

In [10]:
rating=pd.Series(imdb.groupby(['id'])['rating'].mean())

In [11]:
rating=pd.DataFrame(rating)

In [12]:
rating.head()

,rating
id,
1,3.872470
2,3.401869
3,3.161017
4,2.384615
5,3.267857


In [13]:
df=df.merge(rating,how="outer",left_on="id",right_on='id')

In [14]:
df.head()

,original_title,revenue,runtime,budget,popularity,original_language,genres,rating
id,,,,,,,,
5.0,Four Rooms,4300000.0,98.0,4000000.0,9.026586,en,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",3.267857
11.0,Star Wars,775398007.0,121.0,11000000.0,42.149697,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",3.689024
12.0,Finding Nemo,940335536.0,100.0,94000000.0,25.497794,en,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",2.861111
13.0,Forrest Gump,677945399.0,142.0,55000000.0,48.307194,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.937500
14.0,American Beauty,356296601.0,122.0,15000000.0,20.726578,en,"[{'id': 18, 'name': 'Drama'}]",3.451613


In [15]:
df=df.dropna()
df

,original_title,revenue,runtime,budget,popularity,original_language,genres,rating
id,,,,,,,,
5.0,Four Rooms,4.300000e+06,98.0,4000000.0,9.026586,en,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",3.267857
11.0,Star Wars,7.753980e+08,121.0,11000000.0,42.149697,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",3.689024
12.0,Finding Nemo,9.403355e+08,100.0,94000000.0,25.497794,en,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",2.861111
13.0,Forrest Gump,6.779454e+08,142.0,55000000.0,48.307194,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.937500
14.0,American Beauty,3.562966e+08,122.0,15000000.0,20.726578,en,"[{'id': 18, 'name': 'Drama'}]",3.451613
15.0,Citizen Kane,2.321767e+07,119.0,839727.0,15.811921,en,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",2.318182
16.0,Dancer in the Dark,4.003188e+07,140.0,12800000.0,10.684806,en,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",3.948864
18.0,The Fifth Element,2.639202e+08,126.0,90000000.0,24.30526,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",3.288462
19.0,Metropolis,6.504220e+05,153.0,92620000.0,14.487867,de,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",2.597826
